In [1]:
import pandas as pd
import numpy as np
import os
import json
import joblib

import altair as alt
import vegafusion as vf
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
# Pre-define chart function
def chart(df, x, y, title, color=alt.value('steelblue'), width=480, height=320):
    return alt.Chart(df).encode(
        x=x,
        y=y,
        color=color,
    ).properties(
    title=title,
    width=width,
    height=height,
).configure(
    axis=alt.AxisConfig(
        domain=False, # remove axis line
        ticks=False, # remove ticks
        labelAngle=0, # rotate labels
        labelColor='gray', # color of labels
        labelFontSize=10,
    ),
    font='Helvetica Neue',
    view=alt.ViewConfig(stroke=None), # remove border
)

## 1 - Load the training data

In [3]:
input_path = os.path.join('..', 'data', 'cleaned')

train_df = pd.read_parquet(os.path.join(input_path, 'train.parquet'))
train_df.head()

,file,VMONTH,VYEAR,VDAYR,AGE,AGER,SEX,ETHNIC,RACE,USETOBAC,...,PHYS,PHYSASST,NPNMW,RNLPN,OTHPROV,MHP,REFOTHMD,RETAPPT,OTHDISP,ERADMHOS
0,opd2006,12,2006,6,55,4,2,2,1,1,...,1,0,0,0,0,<NA>,0,0,0,0
1,opd2006,11,2006,5,66,5,2,2,1,1,...,1,0,0,0,0,<NA>,0,0,0,0
2,opd2006,11,2006,4,71,5,1,2,1,1,...,1,0,0,0,0,<NA>,1,0,0,0
3,opd2006,11,2006,3,1,1,1,2,1,1,...,1,0,0,0,0,<NA>,0,0,0,0
4,opd2006,11,2006,2,21,2,1,2,1,2,...,1,0,0,0,0,<NA>,0,0,0,0


## 2 - Load the vairiables dictionary and define features for clustering

In [4]:
# Load the variables dictionary
with open(os.path.join(input_path, 'variables.json'), 'r') as f:
    variables = json.load(f)

print(f'Variable Categories:\n')
for category, list in variables.items():
    print(f'{category}')
    print(f'{list}')

Variable Categories:

dateOfVisit
['VMONTH', 'VYEAR', 'VDAYR']
demographics
['AGE', 'AGER', 'SEX', 'ETHNIC', 'RACE', 'USETOBAC']
payment
['PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPER']
visitReason
['INJDET', 'MAJOR', 'RFV1', 'RFV2', 'RFV3']
patientClinicHistory
['SENBEFOR', 'PASTVIS']
vitalSigns
['HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS']
imputedFields
['BDATEFL', 'SEXFL', 'SENBEFL', 'PASTFL']
physicianDiagnoses
['DIAG1', 'DIAG2', 'DIAG3']
differentialDiagnoses
['PRDIAG1', 'PRDIAG2', 'PRDIAG3']
presentSymptomsStatus
['ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
services
['BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE', 'CBC', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'BIOPSY', 'CHLAMYD', 'PAPCONV', 'PAPLIQ', 'PAPUNSP', 'HPVDNA', 'EKG', 'URINE', 'HTTAKE', 'WTTAKE


### 2.1 Defining features for clustering

##### !!! The statistical test result of the features should be referred first

In [5]:
train_df[variables['visitReason']].value_counts()

INJDET  MAJOR  RFV1   RFV2   RFV3 
5       2      48000  44100  41150    40
1       2      23200  48000  44100    25
5       1      10100  14400  14000    21
               14400  14000  14551    21
1       3      46050  47350  11500    20
                                      ..
5       1      14400  15950  15300     1
                             14750     1
                             10200     1
                      15702  41150     1
        5      89980  45650  42050     1
Name: count, Length: 7796, dtype: int64

In [6]:
# Defining the independent variables as features for clustering
features = \
    ['AGE', 'SEX', 'USETOBAC'] + variables['visitReason'] + ['PASTVIS'] + variables['vitalSigns'] \
    + variables['presentSymptomsStatus']

print(f'Features: {features}')
print(f'Number of Features: {len(features)}')

Features: ['AGE', 'SEX', 'USETOBAC', 'INJDET', 'MAJOR', 'RFV1', 'RFV2', 'RFV3', 'PASTVIS', 'HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS', 'ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
Number of Features: 30


In [7]:
clf_df = train_df.loc[:, features]

## 3 - Preprocess and engineer the features

### 3.1 - Binning
Bin the following quantitative variables:

AGE, RFV1, RFV2, RFV3, BMI, TEMPF, BPSYS, BPDIAS

#### 3.1.1 - Bin the AGE variable
Do we bin as recoded age groups (`AGER`) or as each 20 years?

In [9]:
# Check the distribution of `AGER`
# 1 = Under 15 years 
# 2 = 15-24 years 
# 3 = 25-44 years 
# 4 = 45-64 years 
# 5 = 65-74 years 
# 6 = 75 years and over|
 
chart(
    df=train_df,
    x='AGER:O',
    y='count()',
    title='Distribution of AGER',
).mark_bar().interactive()

alt.Chart(...)

In [10]:
# Bin the AGE variable as age groups
# 0-2 = Infant
# 2-4 = Toddler
# 4-12 = Child
# 12-20 = Teenager
# 20-40 = Adult
# 40-60 = Middle Aged
# >= 60 = Senior

age_groups = ['Infant', 'Toddler', 'Child', 'Teenager', 'Child or Teenager', 'Adult', 'Middle Aged', 'Senior']

def bin_age(age):
    #if age < 2: return 'Infant'
    #elif age < 4: return 'Toddler'
    #elif age < 12: return 'Child'
    #elif age < 20: return 'Teenager'
    if age < 20: return 'Child or Teenager'
    elif age < 40: return 'Adult'
    elif age < 60: return 'Middle Aged'
    else: return 'Senior'
    

clf_df['AGE_GROUP'] = clf_df['AGE'].apply(bin_age)

# Check the distribution of age groups
chart(
    df=clf_df,
    x=alt.X('AGE_GROUP:O', sort=age_groups),
    y='count()',
    title='Distribution of AGE GROUPS',
).mark_bar().interactive()

alt.Chart(...)

#### 3.1.2 - Bin the Reason For Visit variables
RFV1, RFV2, RFV3

In [11]:
# Load the REASON FOR VISIT classification summary of codes
rvf_summary = pd.read_excel(os.path.join('..', 'data', 'raw', 'RFV_codes_summary.xlsx'))

# Split the 'CODE NUMBER' column into 'START' and 'END' columns
rvf_summary[['START', 'END']] = rvf_summary['CODE NUMBER'].str.split('-', expand=True).astype(int)

rvf_summary.sample(5)

,MODULE_1,MODULE_2,CODE NUMBER,START,END
34,TREATMENT MODULE,Specific Therapeutic Procedures,4500-4599,4500,4599
41,TEST RESULTS MODULE,TEST RESULTS MODULE,6100-6700,6100,6700
22,DISEASE MODULE,Diseases of the Skin and Subcutaneous Tissue,2800-2899,2800,2899
32,TREATMENT MODULE,Preoperative and Postoperative Care,4200-4299,4200,4299
15,DISEASE MODULE,Diseases of the Nervous System,2350-2399,2350,2399


In [13]:
# Find the `START` and `END` range, 
# and map the corresponding `MODULE_2` to clf_df,
# according to the value of `RFV1`

def get_module(code):
    return rvf_summary[(rvf_summary['START'] <= code) & (rvf_summary['END'] >= code)]['MODULE_2'].values[0]


clf_df['RFV1_MOD'] = clf_df['RFV1'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
print(f'RFV1_MOD unique values: {clf_df["RFV1_MOD"].value_counts()}')

clf_df['RFV2_MOD'] = clf_df['RFV2'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
print(f'RFV2_MOD unique values: {clf_df["RFV2_MOD"].value_counts()}')

clf_df['RFV3_MOD'] = clf_df['RFV3'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
print(f'RFV3_MOD unique values: {clf_df["RFV3_MOD"].value_counts()}')

RFV1_MOD unique values: RFV1_MOD
Progress Visit, NEC                                                  9641
Symptoms Referable to the Musculoskeletal System                     8232
Special Examinations                                                 7416
General Examinations                                                 6895
Symptoms Referable to the Respiratory System                         5908
General Symptoms                                                     4536
Injury by Type and/or Location                                       4312
Symptoms Referable to Psychological and Mental Disorders             4048
Symptoms Referable to the Genitourinary System                       3847
Diagnostic Tests                                                     3801
Symptoms Referable to the Digestive System                           3301
Preoperative and Postoperative Care                                  2971
Symptoms Referable to the Skin, Nails, and Hair                      2845
Sympt

#### 3.1.3 - Bin the vitalSigns variables
BMI, TEMPF, BPSYS, BPDIAS

In [14]:
# Bin the BMI as weight status
# <18.5 = Underweight
# 18.5-25 = Normal weight
# 25-30 = Overweight
# >=30 = Obesity

bmi_groups = ['Underweight', 'Normal weight', 'Overweight', 'Obesity']

def bin_bmi(bmi):
    if bmi < 18.5: return 'Underweight'
    elif bmi < 25: return 'Normal weight'
    elif bmi < 30: return 'Overweight'
    else: return 'Obesity'

clf_df['BMI_GROUP'] = clf_df['BMI'].apply(bin_bmi)

# Check the distribution of BMI groups
chart(
    df=clf_df,
    x=alt.X('BMI_GROUP:O', sort=bmi_groups),
    y='count()',
    title='Distribution of BMI GROUPS',
).mark_bar().interactive()

alt.Chart(...)

In [16]:
# Bin the TEMPF as fever status
# <95 = Hypothermia
# 95-99 = Normal temperature
# 99-100 = Low grade fever
# 100-103 = Fever
# >=103 = Hyperpyrexia

tempf_groups = ['Hypothermia', 'Normal temperature', 'Low grade fever', 'Fever', 'Hyperpyrexia']

def bin_tempf(tempf):
    if tempf < 95: return 'Hypothermia'
    elif tempf < 99: return 'Normal temperature'
    #elif tempf < 100: return 'Low grade fever'
    elif tempf < 103: return 'Fever'
    else: return 'Hyperpyrexia'

clf_df['TEMPF_GROUP'] = clf_df['TEMPF'].apply(bin_tempf)

# Check the distribution of TEMPF groups
chart(
    df=clf_df,
    x=alt.X('TEMPF_GROUP:O', sort=tempf_groups),
    y='count()',
    title='Distribution of TEMPF GROUPS',
).mark_bar().interactive()

alt.Chart(...)

In [17]:
# Bin the BPSYS as systolic blood pressure status
# <90 = Hypotension
# 90-120 = Normal blood pressure
# 120-140 = Prehypertension
# >=140 = Hypertension

bpsys_groups = ['Hypotension', 'Normal blood pressure', 'Prehypertension', 'Hypertension']

def bin_bpsys(bpsys):
    if bpsys < 90: return 'Hypotension'
    elif bpsys < 120: return 'Normal blood pressure'
    elif bpsys < 140: return 'Prehypertension'
    else: return 'Hypertension'

clf_df['BPSYS_GROUP'] = clf_df['BPSYS'].apply(bin_bpsys)

# Check the distribution of BPSYS groups
chart(
    df=clf_df,
    x=alt.X('BPSYS_GROUP:O', sort=bpsys_groups),
    y='count()',
    title='Distribution of BPSYS GROUPS',
).mark_bar().interactive()

alt.Chart(...)

In [18]:
# Bin the BPDIAS as diastolic blood pressure status
# <60 = Low diastolic blood pressure
# 60-90 = Normal diastolic blood pressure
# 90-110 = High diastolic blood pressure
# >=110 = Hypertension

bpdias_groups = [
    'Low diastolic blood pressure', 'Normal diastolic blood pressure', 'High diastolic blood pressure', 'Hypertension'
]

def bin_bpdias(bpdias):
    if bpdias < 60: return 'Low diastolic blood pressure'
    elif bpdias < 90: return 'Normal diastolic blood pressure'
    elif bpdias < 110: return 'High diastolic blood pressure'
    else: return 'Hypertension'

clf_df['BPDIAS_GROUP'] = clf_df['BPDIAS'].apply(bin_bpdias)

# Check the distribution of BPDIAS groups
chart(
    df=clf_df,
    x=alt.X('BPDIAS_GROUP:O', sort=bpdias_groups),
    y='count()',
    title='Distribution of BPDIAS GROUPS',
).mark_bar().interactive()

alt.Chart(...)

### 3.2 - Normalization
Normalize the following quantitative variables:

PASTVIS, HTIN, WTLB

In [8]:
from sklearn.preprocessing import StandardScaler

In [20]:
quantitative_features = ['PASTVIS', 'HTIN', 'WTLB']

In [ ]:
# Get a list of quantitative features
quantitative_features = [feature for feature in features if clf_df[feature].dtype != 'object']

# Normalize quantitative features
scaler = StandardScaler()
clf_df[quantitative_features] = scaler.fit_transform(clf_df[quantitative_features])

# Check the result
clf_df[quantitative_features].head()

### 3.3 - Encode categorical features

In [21]:
# Check binary features
binary_features = [feature for feature in features if clf_df[feature].nunique() == 2]
print(f'Binary Features: {binary_features}')
print(f'Number of Binary Features: {len(binary_features)}')

Binary Features: ['SEX', 'USETOBAC', 'ARTHRTIS', 'ASTHMA', 'CANCER', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
Number of Binary Features: 16


In [ ]:
# Encode binary features


In [ ]:
# Check non-binary categorical features
non_binary_categorical_features = [
    feature for feature in features if clf_df[feature].nunique() > 2 and feature not in quantitative_features
]
print(f'Non-binary Categorical Features: {non_binary_categorical_features}')
print(f'Number of Non-binary Categorical Features: {len(non_binary_categorical_features)}')

In [ ]:
# Encode non-binary categorical features


In [ ]:
# Get a list of categorical features
categorical_features = [feature for feature in features if feature not in quantitative_features]

# One-hot encode categorical features
clf_df = pd.get_dummies(clf_df, columns=categorical_features, drop_first=True)

# Get the list of encoded features
encoded_features = [feature for feature in clf_df.columns if feature not in quantitative_features + categorical_features]

# Check the result
clf_df[encoded_features].head()

### 3.4 - Dimensionality reduction for categorical features

### 3.5 - Redefine the classification DataFrame for training

In [ ]:
# Redefine the classification DataFrame
clf_df = clf_df.loc[:, quantitative_features + encoded_features]

## 4 - Prepare dependent variables

### 4.1 - Employing the hierachical classifications of ICD-9-CM codes to prepare the target labels

In [ ]:
# Check the missing values in 'DIAG1', 'DIAG2', and 'DIAG3'

# Check the distribution of 'DIAG1', 'DIAG2', and 'DIAG3', given the values in 'PRDIAG1', 'PRDIAG2', and 'PRDIAG3'


## 5 - Baseline classification model

### 5.1 - Train models

#### 5.1.1 - KMeans

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
random_seed = 42

In [ ]:
# Impute the missing quantitative values using KNN
n_neighbors = 5
imputer = KNNImputer(n_neighbors=n_neighbors)

clustering_df_imputed = imputer.fit_transform(clustering_df[quantitative_features])

In [ ]:
# Concatenate the imputed quantitative features with the encoded categorical features
clustering_df_imputed = pd.concat([
    pd.DataFrame(clustering_df_imputed, columns=quantitative_features), 
    clustering_df[encoded_features]
], axis=1)

In [ ]:
# Clustering using KMeans
n_clusters = 10
clustering_model = KMeans(n_clusters=n_clusters, random_state=random_seed)
train_df['cluster'] = clustering_model.fit_predict(clustering_df_imputed)

# Check the result
train_df['cluster'].value_counts()

In [ ]:
# Save the model
clustering_model_path = os.path.join('..', 'models')
clustering_model_name = type(clustering_model).__name__

joblib.dump(clustering_model, os.path.join(clustering_model_path, f'{clustering_model_name}.joblib'))

### 5.2 - Evaluation of the model

In [ ]:
import altair as alt
import vegafusion as vf
alt.data_transformers.enable("vegafusion")

#### 5.2.1 - Metrics

In [ ]:
# Calculate the Silhouette Score
silhouette = silhouette_score(clustering_df_imputed, train_df['cluster'])
print(f'Silhouette Score: {silhouette}')

In [ ]:
# Calculate the sum of squared distances of samples to their closest cluster center
print(f'Sum of Squared Distances: {clustering_model.inertia_}')

#### 5.2.2 - Examine cluster centroids

In [ ]:
# Check the cluster centers
cluster_centers = pd.DataFrame(clustering_model.cluster_centers_, columns=clustering_df_imputed.columns)

# Inverse transform the cluster centers
cluster_centers[quantitative_features] = scaler.inverse_transform(cluster_centers[quantitative_features])

# Check the result
cluster_centers

#### 5.2.3 - Visualization

## 6 - Extract text features from each cluster

#### 6.1 - Aggregate text data by group

In [ ]:
for i, (k, v) in zip (range(len(variables)), variables.items()):
    if i < 5:
        print(k, v)

In [ ]:
for i, (k, v) in zip (range(len(variables)), variables.items()):
    if i >= 5 and i < 10:
        print(k, v)

In [ ]:
for i, (k, v) in zip (range(len(variables)), variables.items()):
    if i >= 10:
        print(k, v)

In [ ]:
train_df.USETOBAC.unique()

In [ ]:
train_df[variables['visitReason']].head()

In [ ]:
train_df[variables['vitalSigns']].value_counts()

In [ ]:
train_df[variables['physicianDiagnoses']].head()

In [ ]:
print(train_df.PRDIAG1.unique())
print()

train_df.DIAG1[train_df.PRDIAG1.str.contains('not probable') | (train_df.PRDIAG1 == 'No')].value_counts()

In [ ]:
train_df[variables['presentSymptomsStatus']].value_counts()

In [ ]:
# Load custom function to combine text features
import sys
sys.path.append('../src/features/')

from combine_textual import combine_features


# Define the list of textual features to combine
textual_features = [
    'AGE', 'SEX', 'USETOBAC', 
    'MAJOR', 'RFV1', 'RFV2', 'RFV3', 
    'BMI', 'TEMPF', 'BPSYS', 'BPDIAS',
    'ARTHRTIS', 'ASTHMA', 'CANCER', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'DMP',
    'DIAG1', 'DIAG2', 'DIAG3'
]

# Export the list of textual features
with open(os.path.join(file_path, 'textual_features.json'), 'w') as f:
    json.dump(textual_features, f)

# Combine the text features
train_df['CombinedText'] = train_df.apply(lambda x: combine_features(x, textual_features), axis=1)

train_df.CombinedText.head()

In [ ]:
train_df.CombinedText.notna().sum()

#### 6.2 - Preprocess text data

In [ ]:
import spacy
import re

In [ ]:
nlp = spacy.load('en_core_web_sm')

def preprocess_text(row):
    row = re.sub(r'(\d+),(\d+)', r'\1\2', row)
    row = re.sub(r'(\d+)-(\d+)', r'\1_\2', row)
    doc = nlp(row)
    processed_text = ' '.join(token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct)
    row = re.sub(r'(\d+)_(\d+)', r'\1-\2', row)
    return processed_text

In [ ]:
train_df['ProcessedText'] = train_df['CombinedText'].apply(preprocess_text)

In [ ]:
train_df.ProcessedText.head()

In [ ]:
# Save the preprocessed DataFrame
processed_file_path = os.path.join('..', 'data', 'processed')
train_df.to_csv(os.path.join(processed_file_path, f'train_{clustering_model_name}.csv'), index=False)

#### 6.3 - Calculate term frequencies

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Method 1
# Calculate the TF-IDF for each cluster,
# taking the ProcessedText of each cluster as the documents,
# and the ProcessedText of the entire dataset as the corpus

#clustered_text = train_df.groupby('cluster')['ProcessedText'].apply(lambda row: ' '.join(row)).reset_index()

#vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=5, max_df=0.7)
#tfidf_matrix = vectorizer.fit_transform(clustered_text['ProcessedText'])

#print(tfidf_matrix)

#tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
#tfidf_df

In [ ]:
# Method 2
# Calculate the TF-IDF of each row within each cluster
# Calculate the average TF-IDF for each cluster

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=5, max_df=0.7)
tfidf_matrix = vectorizer.fit_transform(train_df['ProcessedText'])

# Calculate the average TF-IDF for each cluster
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
cluster_tfidf = pd.concat([train_df['cluster'], tfidf_df], axis=1).groupby('cluster').mean()

cluster_tfidf

In [ ]:
# Punish the weight of '\d+_year_old' by multiplying it by 0.5, using the regex pattern

#tfidf_df = tfidf_df.apply(lambda row: row * 0.5 if re.match(r'\d+_year_old', row.name) else row)
#cluster_tfidf = cluster_tfidf.apply(lambda row: row * 0.5 if re.match(r'\d+_year_old', row.name) else row)

#cluster_tfidf

#### 6.4 - Generate word clouds for each group

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Plot the word cloud for each cluster basd on the average TF-IDF
for i in range(n_clusters):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(cluster_tfidf.loc[i])
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Cluster {i}')
    plt.show()

## 7 - Topic modeling

In [ ]:
import gensim
from gensim import corpora

random_seed = 42

In [ ]:
# Generate bigrams
def generate_bigrams(row):
    tokens = row.split()
    bigram_model = gensim.models.Phrases(tokens, min_count=5, threshold=100)
    tokens = [token for token in bigram_model[tokens]]
    return tokens


print(generate_bigrams(train_df['ProcessedText'].iloc[0]))
preprocessed_docs = train_df['ProcessedText'].apply(generate_bigrams)

In [ ]:
# Build dictionary and corpus
dictionary = corpora.Dictionary(preprocessed_docs)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]

In [ ]:
# Train LDA model
n_topics = 10

lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=n_topics,
    random_state=random_seed,
    chunksize=100,
    passes=10,
)



### 7.1 - Visualize the topics

In [ ]:
import pyLDAvis.gensim_models
import pyLDAvis

pyLDAvis.enable_notebook()

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

### 7.2 - Get the topic distribution for each cluster

In [ ]:
# Get the topic distribution for each document 
def get_avg_topic_distribution(cluster_data, topic_distribution, num_topics):
    cluster_topics = np.zeros((len(cluster_data.index), num_topics))

    # Update the distribution with the actual values
    for i, doc_index in enumerate(cluster_data.index.tolist()):
        for topic, prob in topic_distribution[doc_index]:
            cluster_topics[i, topic] = prob

    avg_topic_dist = np.mean(cluster_topics, axis=0)
    return avg_topic_dist


topic_distribution = lda_model.get_document_topics(corpus, minimum_probability=0.0)
clusters = train_df.groupby('cluster').ProcessedText

cluster_topics = []
for cluster, data in clusters:
    avg_topic_dist = get_avg_topic_distribution(data, topic_distribution, lda_model.num_topics)
    cluster_topics.append([avg_topic_dist.tolist()])

cluster_topics_df = pd.DataFrame(cluster_topics, columns=['avg_topic_distribution'])

cluster_topics_df

### 7.3 - Visualize the distribution of topics within each cluster

In [ ]:
import altair as alt

In [ ]:
# Reshape the dataframe to have a row for each cluster and topic
cluster_topics_heatmap_df = cluster_topics_df.avg_topic_distribution.apply(pd.Series).reset_index().rename(columns={'index': 'cluster'}).melt(id_vars='cluster', var_name='topic', value_name='probability')

heatmap = alt.Chart(cluster_topics_heatmap_df).mark_rect().encode(
    x='topic:O',
    y='cluster:O',
    color='probability:Q'
).properties(
    title='Average Topic Distribution for Each Cluster',
    width=400,
    height=400
)

heatmap